<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/gunpoint_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from torch import nn, optim
from google.colab import drive
drive.mount('/content/drive')
import torch.nn.functional as F
from tqdm import tqdm
import time
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks")
from binary_train import *

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 파일 경로
train_file_path = '/content/drive/MyDrive/Colab Notebooks/data/GunPointOldVersusYoung_TRAIN.txt'
test_file_path = '/content/drive/MyDrive/Colab Notebooks/data/GunPointOldVersusYoung_TEST.txt'

# 데이터셋 클래스 정의
class GunPointDataset(Dataset):
    def __init__(self, file_path, scaler=None):
        # 데이터와 레이블을 로드
        data = np.loadtxt(file_path)
        self.labels = data[:, 0].astype(int) - 1  # 레이블을 0과 1로 조정
        self.features = data[:, 1:]

        if scaler:
            self.features = scaler.fit_transform(self.features)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # 특정 인덱스의 샘플과 레이블을 반환
        return torch.tensor(self.features[idx], dtype=torch.float), self.labels[idx]

scaler = StandardScaler() # data를 스케일링하지 않는경우 초기 모델의 출력값의 폭이 아주 큼.

# 데이터셋 인스턴스 생성
train_dataset = GunPointDataset(train_file_path, scaler)
test_dataset = GunPointDataset(test_file_path, scaler)

# 데이터를 훈련 데이터와 검증데이터로 나누기.
train_size = int(0.9 * len(train_dataset))  # 전체 데이터의 90%를 훈련 데이터로 사용
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# 데이터 로더 생성
batch_size = 4
train_DL = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_DL = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
test_DL = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# 데이터 로더를 통해 데이터 확인
for features, labels in train_DL:
    print(features.shape, labels.shape)
    print(f"Features: {features}, Labels: {labels}")
    break  # 첫 배치만 출력하고 멈춤

In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()

        # 히든레이어의 노드가 10개만 돼도 금방수렴함.

        self.linear = nn.Sequential(nn.Linear(150, 10), nn.ReLU(),
                                    nn.Linear(10,1), nn.Sigmoid(),)

    def forward(self, x):

        x = self.linear(x)

        return x

In [22]:
model = MLP()
LR = 1e-3 # -1 인경우 한번에 0또는 1만 출력하는 로컬 미니멈으로 수렴.
optimizer = optim.Adam(model.parameters(), lr = LR)

In [18]:
Test(model, test_DL)


Test loss: 0.78705
Test accuracy: 165/315 (52.4 %)


52.4

In [24]:
Train(model, train_DL, val_DL, EPOCH = 5, optimizer = optimizer)

train loss: 0.45884, val loss: 0.28881 
train acc: 89.3 %, val acc: 100.0 %, time: 0 s
--------------------


train loss: 0.24117, val loss: 0.16688 
train acc: 88.5 %, val acc: 100.0 %, time: 0 s
--------------------


train loss: 0.17369, val loss: 0.12503 
train acc: 95.9 %, val acc: 100.0 %, time: 0 s
--------------------


train loss: 0.13117, val loss: 0.09665 
train acc: 98.4 %, val acc: 100.0 %, time: 0 s
--------------------


train loss: 0.09561, val loss: 0.07384 
train acc: 99.2 %, val acc: 100.0 %, time: 0 s
--------------------


{'train': [0.45883510738122657,
  0.24117406408806316,
  0.1736948327695737,
  0.1311722901939857,
  0.09561138694770023],
 'val': [0.2888101211615971,
  0.16687895197953498,
  0.1250252319233758,
  0.09664680090333734,
  0.07383794497166361]}